In [1]:
import igraph as ig
import numpy as np
from sympy.solvers import nsolve
from sympy import *

In [19]:
from scipy.stats import rankdata

In [18]:
from matplotlib.font_manager import FontProperties
import matplotlib.gridspec as gridspec

In [2]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [3]:
from scipy.stats import norm

In [4]:
from __future__ import division

In [5]:
import powerlaw as pl
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
from sympy.solvers import nsolve
from sympy import *

In [7]:
from scipy import special
from scipy import stats
import pandas as pd

In [8]:
import scipy as sp
from scipy import stats

In [9]:
from scipy.spatial import distance
def dist(v1,v2):
    dist = distance.euclidean((v1[0],v1[1]),(v2[0],v2[1]))
    return dist

In [13]:
skdata = pd.read_csv('final_table_galicia.csv') ####

In [14]:
sreal = skdata['s'].tolist()

In [15]:
x0_cf = [] ### fixed points
y0_cf = []### fixed points
c=[]
a_12 = [] #### cross-diffusion termns
a_21 = [] #### cross-diffusion termns

for i in range(20):
    if sreal[i] > np.mean(sreal):
        x0_cf.append(1.12)
        y0_cf.append(0.16)
        c.append(0.75)
        a_12.append(1.)
        a_21.append(0.0)
    if sreal[i] <= np.mean(sreal):
        x0_cf.append(0.16)
        y0_cf.append(1.12)
        c.append(-0.75)
        a_12.append(0.0)
        a_21.append(1.)

In [16]:
### add to table
skdata['c'] = c
skdata['x0_cf'] = x0_cf
skdata['y0_cf'] = y0_cf
skdata['a21'] = a_21
skdata['a12'] = a_12

In [ ]:
x_coords = (sdata['x'].values).reshape(550,1)
y_coords = (aggragate['Y'].values).reshape(550,1)
population = ((aggragate['population_y'].values).reshape(550,1)).tolist()
coords = np.concatenate([x_coords,y_coords],axis=1)

In [ ]:
n = len(x_coords)
alldist = []
A = np.zeros((n,n))
##### create the adcecancy matrix with edgeweights from Gravity Law
count=0
for i in range(0,n):
    for j in range(i+1,n):
        m = (population[i][0]*population[j][0])/(dist(coords[i],coords[j]))**2
        A[i][j] = m
        A[j][i] = m
        h = (np.random.binomial(550, 0.011496438001083215, 1))/549 #######create a random network with binomial weights and  
        
        A_binom[j][i] = h
        A_binom[i][j] = h

In [ ]:
np.random.shuffle(list_weight)  #### shuffle the weights to create a random network
A_random = np.zeros((nn,nn))
for i in range(0,nn):
    for j in range(i+1,nn):
        y = list_weight.pop()
        A_random[i][j] = y
        A_random[j][i] = y

In [ ]:
###Define the graph laplacian

Dt = np.sum(A,axis=1).tolist()
DD = np.diag(Dt)
L = A - DD

In [ ]:
###Define the graph laplacian for random network
Dt_random = np.sum(A_random,axis=1).tolist()
DD_random = np.diag(Dt_random)
L_random = A_random - DD_random

In [ ]:
###Define the graph laplacian for random network
Dt_binom = np.sum(A_binom,axis=1).tolist()
DD_binom = np.diag(Dt_binom)
L_binom = A_binom - DD_binom

In [ ]:
cc = (skreal['c'].values).reshape(550,1)  ### language status
a_12 = (skreal['a12'].values).reshape(550,1)
a_21 = (skreal['a21'].values).reshape(550,1)

In [ ]:
### solve the differential equation numerically

def fun(e,f):
    l1 = np.matmul(L,e)
    l5 = np.matmul(L,f)
    l3 = np.matmul(L,(e*f))
    return (e + deltat*(e*(1-e) + cc*e*f + l1*d1 + l3*a_12), f + deltat*(f*(1-f) - cc*e*f + l5*d2 + l3*a_21))

In [ ]:
#### diffusion terms

d1 = 0.025 
d2 = 0.025

In [ ]:
t= 80000#Pasos temporales totales
tiempo=np.linspace(0,t,t+1) 

In [ ]:
#### run diffusion over the geographical network of Galicia
### run multiple realiaztions
evolution2x = []
evolution2y = []
for i in range(50):
    up = []
    vp = []
    x_0 = (aggragate['x0_cf'].values).reshape(550,1)
    y_0  = (aggragate['y0_cf'].values).reshape(550,1)
    for n in range(len(population)):
        up.append(x_0[n] + 0.1*x_0[n]*(2.*np.random.rand()-1))
        vp.append(y_0[n] + 0.1*y_0[n]*(2.*np.random.rand()-1))
    upp = np.array([up]).T
    vpp = np.array([vp]).T
    ui = upp
    vi = vpp
    uavg= []
    for j in tiempo:
        u =(fun(ui,vi))[0]
        v=(fun(ui,vi))[1]
        ui = u
        vi = v
        uavg.append(np.average(ui))
    evolution2x.append(ui)
    evolution2y.append(vi)
    print(i)

In [ ]:
### take the average of multiple runs
skreal['xi_sim'] = np.mean(evolution2x,axis=0)[0]
skreal['yi_sim'] = np.mean(evolution2y,axis=0)[0]

In [17]:
new=pd.DataFrame()

In [ ]:
new['xi_n'] = skreal.groupby('Region').apply(lambda x: np.average(x['xi_n']/(x['xi_n'] + x['yi_n']), weights=x['population_y'])).tolist()
new['yi_n'] = skreal.groupby('Region').apply(lambda x: np.average(x['yi_n']/(x['xi_n'] + x['yi_n']), weights=x['population_y'])).tolist()

In [ ]:
###compare with actual data
compare = pd.merge(new,skdata,left_on='Comarca',right_on='comarca')

In [ ]:
compare['g_n'] = np.array(compare['galicia'])/np.array(compare['galicia'] + compare['spanish'])

In [ ]:
compare['s_n'] = np.array(compare['spanish'])/np.array(compare['galicia'] + compare['spanish'])

In [20]:
font0 = FontProperties()
font1 = font0.copy()
font1.set_weight('bold')
font1.set_size('large')
font1.set_family('sans-serif')

In [ ]:

### plot simulation concentration vs real concentration of speakers
fig, axes = plt.subplots(2,2, figsize=(2*6.4, 2*4.8))

axes[0][0].plot(compare['xi_n'].tolist(),compare['g_n'].tolist(),'o',color='black')
axes[0][0].set_xlim(-0.05,1.05)
axes[0][0].set_ylim(-0.05,1.05)
axes[0][0].tick_params(axis='both', which='major', labelsize=15)
axes[0][0].set_xlabel('Simulation Concentration',fontsize = 15)
axes[0][0].set_ylabel('Real Concentration',fontsize=15)
axes[0][0].text(0.7,0.2,r'$\rho^G_p = 0.77$',fontsize=20)
axes[0][0].text(0.05, 0.95, '(a)',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[0][0].transAxes)

axes[0][1].plot(compare['yi_n'].tolist(),compare['s_n'].tolist(),'o',color='black')
axes[0][1].set_xlim(-0.05,1.05)
axes[0][1].set_ylim(-0.05,1.05)
axes[0][1].tick_params(axis='both', which='major', labelsize=15)
axes[0][1].set_xlabel('Simulation Concentration',fontsize = 15)
axes[0][1].set_ylabel('Real Concentration',fontsize=15)
axes[0][1].text(0.05, 0.95, '(b)',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[0][1].transAxes)
axes[0][1].text(0.7,0.2,r'$\rho^S_p = 0.77$',fontsize=20)

axes[1][0].plot(rankdata(-np.array(compare['xi_n'].tolist())),rankdata(-np.array(compare['g_n'].tolist())),'o',color='black')
axes[1][0].set_xscale('log')
axes[1][0].set_yscale('log')
axes[1][0].set_xlim(100,0.1)
axes[1][0].set_ylim(100,0.1)
axes[1][0].set_xlabel('Simulation Rank',fontsize = 15)
axes[1][0].set_ylabel('Real Rank',fontsize=15)
axes[1][0].tick_params(axis='both', which='major', labelsize=15)
axes[1][0].text(0.05, 0.95, '(c)',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[1][0].transAxes)
axes[1][0].text(5,0.2,r'$\rho^G_s = 0.85$',fontsize=20)

axes[1][1].plot(rankdata(-np.array(compare['yi_n'].tolist())),rankdata(-np.array(compare['s_n'].tolist())),'o',color='black')
#axes[1][0].set_xlim(-0.05,1.05)
#axes[1][0].set_ylim(-0.05,1.05)
axes[1][1].set_xscale('log')
axes[1][1].set_yscale('log')
axes[1][1].set_xlim(100,0.1)
axes[1][1].set_ylim(100,0.1)
axes[1][1].set_xlabel('Simulation Rank',fontsize = 15)
axes[1][1].set_ylabel('Real Rank',fontsize=15)
axes[1][1].tick_params(axis='both', which='major', labelsize=15)
axes[1][1].text(0.05, 0.95, '(d)',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[1][1].transAxes)
axes[1][1].text(10,0.2,r'$\rho^S_s = 0.85$',fontsize=20)

plt.tight_layout()

plt.savefig('fig_s4.pdf',bbox_inches='tight')